# TF-IDF
Recapitulando un poco nuestro proyecto, queremos que dada una subcadena encontrar algun subtitulo que contenga dicha subcadena. 
TF-IDF nos permite buscar entre una gran cantidad de documentos a diferencia de RAKE. Además nos permite obtener un ejemplo mas representativo de dicha subcadena, pues TF-IDF le da un valor a la representatividad.
TF se encarga de encontrar la frecuencia de palabras dentro de un documento e IDF se encarga de revisar las estas frecuencias y avergiuar que tan representativas son para cada documento (video).

Si acotamos el problema a meramente coincidencias de las palabras no necesitamos ningun espacio vectorial porque no nos interesa la semantica de las palabras. Pues queremos coincidencias exactas y no frases con una semantica similar.

Tambien, si decidieramos implementar un espacio vectorial, podriamos hacer uso de vectores TF-IDF. Tomando en cuenta que nuestros documentos seran nuestros subtitulos en lugar de tomar cada video como un documento.

In [27]:
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords #Listas de stopwords
from nltk.tokenize import word_tokenize#Tokens
import re #regex
import pickle
from tqdm import tqdm 
from collections import defaultdict, Counter
import pandas as pd
from operator import itemgetter
import numpy as np

[nltk_data] Downloading package punkt to /home/karla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
with open("../pkl/clean_videos.pkl", "rb") as f:
    videos = pickle.load(f)
videos[0][0]

{'id': 'L9YhoRatRzE',
 'original_title': 'Siempre Fui Yo | Adelanto | Disney+',
 'subtitles': [{'start': '0.13',
   'dur': '3.77',
   'text': ['tu', 'papá', 'tuvo', 'un', 'accidente']},
  {'start': '12.5', 'dur': '5.939', 'text': ['te', 'recuerdo', 'que', 'está']},
  {'start': '15.59', 'dur': '5.339', 'text': ['aquí']},
  {'start': '18.439',
   'dur': '6.361',
   'text': ['estaba', 'como', 'rabioso', 'con']},
  {'start': '20.929',
   'dur': '7.65',
   'text': ['especial', 'con', 'lucas', 'martín']},
  {'start': '24.8',
   'dur': '3.779',
   'text': ['necesito', 'saber', 'qué', 'fue', 'lo', 'que', 'pasó']},
  {'start': '29.42', 'dur': '2.479', 'text': ['aplausos']}]}

In [8]:
documents = defaultdict(list)
def merge_videos(corpus):
    '''
    Mezcla el corpus de cada video en una entrada del
    diccionario movies en un solo corpus, 
    guarda los diálogos por línea tokenizados y limpios

    Args:
        col (dic): Diccionario con los diálogos
    '''
    corpus_clean = []
    #Iteramos sobre los canales
    for chanel in tqdm(corpus):
        for video in chanel:
            #Cada texto se guarda por oraciones
            if 'subtitles' in video:
                corpus_clean += [s['text'] for s in video['subtitles']]
                for s in video['subtitles']:
                    documents[video['id']] += s["text"]
    return corpus_clean

all_videos = merge_videos(videos)
all_videos[0]

100%|██████████| 9/9 [00:00<00:00, 16.92it/s]


['tu', 'papá', 'tuvo', 'un', 'accidente']

In [11]:
def flatten(lst):
    return [item for sub in lst for item in sub]

In [12]:
term_frequencies = Counter(flatten(all_videos))

In [25]:
term_list = pd.DataFrame(sorted(term_frequencies.items(), key=itemgetter(1), reverse=True), 
                         columns=['Token','Frequency'])

term_list = term_list.set_index(term_list['Token'])
term_list.pop('Token')
term_list

,Frequency
Token,
de,217897
que,167644
la,128609
y,116389
el,105011
...,...
nananana,1
nía,1
llull,1


In [68]:
idf = {}
num_documents = len(documents)
for term in tqdm(term_list.index):
    #Total de documentos en que
    #aparece el término
    total_documents = 0
    for doc, token_list in documents.items():
        if term in token_list:
            total_documents += 1
            
    #Asignación de idf
    idf[term] = -np.log2(total_documents/num_documents)

#Agregamos el idf al dataframe
term_list['idf'] = idf.values()
term_list = term_list.sort_values(by='idf', ascending=False)
    
pickle.dump(term_list, open("../pkl/term_list.pkl", "wb"))

100%|██████████████████████████████████| 97791/97791 [3:54:51<00:00,  6.94it/s]


In [9]:
documents["L9YhoRatRzE"]

['tu',
 'papá',
 'tuvo',
 'un',
 'accidente',
 'te',
 'recuerdo',
 'que',
 'está',
 'aquí',
 'estaba',
 'como',
 'rabioso',
 'con',
 'especial',
 'con',
 'lucas',
 'martín',
 'necesito',
 'saber',
 'qué',
 'fue',
 'lo',
 'que',
 'pasó',
 'aplausos']

In [20]:
with open("../pkl/term_list.pkl", "rb") as f:
    term_list = pickle.load(f)

term_list = term_list.sort_values(by='idf', ascending=False)
term_list[100:130]

,Frequency,idf
Token,,
blass,5,11.67904
neocelandés,1,11.67904
juventudes,1,11.67904
jarryd,1,11.67904
première,1,11.67904
yesin,1,11.67904
fosis,1,11.67904
identifique,1,11.67904
contemplaban,1,11.67904


In [26]:
#Parametro
a = 0.5

#Guarda los tfidf
tfidf = {}
for doc, token_list in documents.items():
    try:
        #Cuenta la frecuencia del término en el documento
        term_freq_doc = Counter(token_list)
        #print(term_freq_doc.values())
        #Obtiene la frecuencia mayor
        max_freq = np.max(list(term_freq_doc.values()))
        #Guarda tfidf por documento
        tfidf_doc = {}
        for term in term_freq_doc:
            #Valor de TF
            tf = a + (1-a)*(term_list["Frequency"][term]/max_freq)
            #Valor de tfidf
            tfidf_doc[term] = tf*term_list["idf"][term]
            
        tfidf[doc] = tfidf_doc
    except:
        print(doc,token_list)

jBHKT7_GFqU []
p-rjN7NfKw4 []


In [24]:
tfidf["L9YhoRatRzE"]

{'tu': 1418.7040158067475,
 'papá': 413.2296042991872,
 'tuvo': 433.0538386324496,
 'un': 1977.6454572285645,
 'accidente': 178.37919007682308,
 'te': 1675.3258637158917,
 'recuerdo': 306.48387045316315,
 'que': 2754.05210974035,
 'está': 1444.7053994278488,
 'aquí': 1349.9768500097691,
 'estaba': 996.2010785976058,
 'como': 2163.693302579083,
 'rabioso': 16.37494616013497,
 'con': 1854.840635821252,
 'especial': 503.554450076275,
 'lucas': 23.400066758614557,
 'martín': 80.14848223671673,
 'necesito': 322.01821137768957,
 'saber': 592.050234798179,
 'qué': 1366.1480148444505,
 'fue': 1524.2868763932865,
 'lo': 1667.7345204412673,
 'pasó': 526.0974232026272,
 'aplausos': 771.3768033282756}

In [25]:
pickle.dump(tfidf, open("../pkl/tfidf.pkl", "wb"))

## Recuperación de Documentos con TFIDF

In [42]:
# tokens que necesitan ser limpiados del corpus y que no
# se encuentran en la lista de stopwords
more = ["si", "bien", "ahora", "así", "aquí", "pues"]
stopwords_list = stopwords.words('spanish') + more

def get_tokens_clean(text, stop_words=True):
    '''
    Genera los tokens de una cadena y los limpia
    (quita símbolos raros y stopwords)
    
    Args:
        text (str): cadena
    '''
    tokens = word_tokenize(text)
    clean = []
    pattern = r'[-_{}(),;:"#\/.¡!¿?·\[\]\'`]'
    for w in tokens:
        #quita stopwords y convierte a minúsculas
        w = re.sub(pattern,'', w.lower())
        if w != '':
            clean.append(w)
        """if w not in stopwords_list and w != '':
            clean.append(w)
        elif not stop_words and w != '':
            clean.append(w)"""
    return clean

In [48]:
def recover_documents(query, tfidf_weights):
    """
    Función para recuperar documentos en base a su valor de TFIDF.
    
    Arguments
    ---------
    query : str
        Cadena a buscar
    tfidf_weights : dict
        Diccionario de tfidf por documentos y términos
    """
    #Obtener términos en query y procesarlos
    proc_query = flatten([get_tokens_clean(w) for w in query.split()])
    print(proc_query)
    for docID, tfidf_doc in tfidf_weights.items():
        #Guarda el score
        score = 0
        for w in proc_query:
            #Revisa si todos los terminos de la query están en el documento
            if w in tfidf_doc.keys():
                #Suma el score
                score += tfidf_doc[w]
            else:
                score = 0
                break
        yield docID, score

In [49]:
#Consulta
query = 'tengo hambre'
#Resultados de la consulta
results = recover_documents(query, tfidf)
#Ordena la consulta de mayor peso a menor
sorted_results = sorted(results, key=itemgetter(1), reverse=True)
print(len(sorted_results))
for docID, score in sorted_results:
    print('docID: {}, score: {}'.format(docID, score))

['tengo', 'hambre']
3276
docID: QbsR17gqWvg, score: 512.260868332419
docID: k_ircrGCqV8, score: 172.5918680665185
docID: xQXmg3HzkxQ, score: 141.71286804234572
docID: p5vm5sO5zvY, score: 135.67132455935538
docID: 49ZjRErUkyI, score: 120.33509879484147
docID: TewiP_1y3BM, score: 120.33509879484147
docID: kiI5hQIpNCA, score: 120.33509879484147
docID: FsiRGY7iHKw, score: 98.28927425835275
docID: TEKmSbPbcBg, score: 95.39436800608655
docID: JLZkH87R3GU, score: 87.67461800004335
docID: sGsPiHRWN2U, score: 87.67461800004335
docID: Olo7mD19aEY, score: 85.37955718743592
docID: vjW0UXiga48, score: 83.20528904917623
docID: cwYKk-jQ63o, score: 81.14252184108372
docID: QWOCO8bgJeI, score: 65.14555165587647
docID: DWABONpr-90, score: 63.89778798143031
docID: U-lHf5SHAk8, score: 58.33956797707921
docID: Duyqu1082M0, score: 53.7077179734533
docID: 85lf6HGgW7c, score: 52.872466333455186
docID: d8UyNk5YzMk, score: 52.06415829474733
docID: 4UonMu7jB4w, score: 51.28151082869687
docID: mpE1u-NFD48, score:

Levenshtein con palabras en lugar de carácteres con los documentos que ya recuperamos para regresar los subtitulos más parecidos a la query